In [ ]:
# Instalar ipympl o cambiar por %matplotlib notebook
%matplotlib widget
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display,clear_output
import matplotlib.pyplot as plt

In [ ]:
paises_ocde = ['Australia', 'Austria', 'Belgium', 'Canada', 'Chile', 'Colombia',
       'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany',
       'Greece', 'Hungary', 'Iceland', 'Ireland', 'Israel', 'Italy',
       'Japan', 'Korea, South', 'Latvia', 'Lithuania', 'Luxembourg',
       'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland',
       'Portugal', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
       'Switzerland', 'Turkey', 'US', 'United Kingdom']

In [ ]:
df_confirmados = pd.read_csv('data/Confirmados_OCDE.csv',index_col='Country/Region')
df_recuperados = pd.read_csv('data/Recuperados_OCDE.csv',index_col='Country/Region')
df_muertes = pd.read_csv('data/Muertes_OCDE.csv',index_col='Country/Region')

poblacion_total = pd.read_csv('data/Población_OCDE.csv',index_col='Unnamed: 0')['Population (2020)'].astype('float64')

df_confirmados.rename(mapper=pd.to_datetime, axis='columns', inplace=True);
df_recuperados.rename(mapper=pd.to_datetime, axis='columns', inplace=True);
df_muertes.rename(mapper=pd.to_datetime, axis='columns', inplace=True);

In [ ]:
# UI Setup

# Dropdowns

dropdownA = widgets.Dropdown(
    options=paises_ocde,
    value='Canada',
    description='Países',
    style = {'description_width':'102px'},
    layout={'width': '310px'}
)

dropdownB = widgets.Dropdown(
    options=paises_ocde,
    value='Chile',
    description='',
    layout={'width': '200px'}
)

dropdownsLayout = widgets.HBox([dropdownA,dropdownB])

# Toggles
toggleStyles = {'button_width':'200px','button_width':'100px'}

modoToggle = widgets.ToggleButtons(
    options=['Casos Diarios','Casos Acumulados'],
    description='Modo:',
    button_style='success',
    style={'button_width':'200px','description_width':'100px'},
    tooltips=['Muestra los casos nuevos de cada día o semana','Muestra los casos acumulados hasta la fecha']
)

normToggle = widgets.ToggleButtons(
    options=['Valor Absoluto', 'Valores Relativos'],
    description='Normalización:',
    button_style='info',
    style = modoToggle.style,
    tooltips=['Valor absoluto', 'Tasas por 100.000 habitantes']
)

freqToggle = widgets.ToggleButtons(
    options=['Valores Diarios', 'Valores Semanales'],
    description='Agrupamiento:',
    button_style='warning',
    style = modoToggle.style,
    tooltips=['Muestra valores diarios', 'Agrupa valores por semana ( de lunes a lunes )']
)

# Layout principal
mainLayout = widgets.VBox([dropdownsLayout,modoToggle,normToggle,freqToggle])

# Tabs
tabs = widgets.Tab()
tab_names = ['Confirmados','Recuperados','Decesos']

for i in range(3):
    tabs.set_title(i, tab_names[i])
    
tabs.children = [mainLayout,mainLayout,mainLayout]

# Layout con Output
out = widgets.Output()
outLayout = widgets.VBox([tabs,out])

In [ ]:
# Arreglo de tiempo por día
with out:
    plt.figure(figsize=(9, 5),tight_layout=True)
    plt.figure(1).canvas.header_visible = False
def plot():
    with out:
        
        df = None
        plt.cla()
        
        # Confirmados
        if tabs.selected_index == 0:
            df = df_confirmados
            plt.title('Confirmados')
        # Recuperados
        elif tabs.selected_index == 1:
            df = df_recuperados
            plt.title('Recuperados')
        # Decesos
        else:
            df = df_muertes
            plt.title('Decesos')
        
        
        dfA = df.loc[dropdownA.value].copy()[1:]
        dfB = df.loc[dropdownB.value].copy()[1:]
        
        if modoToggle.value == 'Casos Diarios':
            dfA = dfA - dfA.shift(1)
            dfB = dfB - dfB.shift(1)
            
        if normToggle.value == 'Valores Relativos':
            dfA = (100000.0*dfA)/poblacion_total[dropdownA.value]
            dfB = (100000.0*dfB)/poblacion_total[dropdownB.value]
            plt.ylabel('Tasa por 100.000 habitantes')
        else:
            plt.ylabel('Número de Casos')
            
        if freqToggle.value == 'Valores Semanales':
            dfA = dfA.T.groupby(pd.Grouper(freq='W-MON')).sum()
            t = dfA.index.to_numpy()
            dfA = dfA.to_numpy()
            dfB = dfB.T.groupby(pd.Grouper(freq='W-MON')).sum()
            plt.xlabel('Semana')
        else:
            t = df_confirmados.columns.to_numpy()[1:]
            plt.xlabel('Día')
        
        
        plt.plot_date(t, dfA, '#3a86ff', label=dropdownA.value)
        plt.plot_date(t, dfB, '#fb5607', label=dropdownB.value)
        plt.legend()
        plt.show()

# Dashboard COVID-19 OCDE

Casos confirmados, recuperados y decesos hasta el 26 de Abril de 2021.

In [ ]:
# Interacción
display(outLayout)
    
# Actualiza series
def updateSeries(a,b,c,d,e):
    plot()
    
prevIndex = 0
def updateTab():
    global prevIndex
    if prevIndex != tabs.selected_index:
        plot()
    prevIndex = tabs.selected_index
    
widgets.interactive_output(updateSeries,{
    'a':dropdownA,
    'b':dropdownB,
    'c':modoToggle,
    'd':normToggle,
    'e':freqToggle
})
tabs.observe(lambda change: updateTab(), names='selected_index')